In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Define the neural network
class TwistedMNISTModel(nn.Module):
    def __init__(self):
        super(TwistedMNISTModel, self).__init__()
        
        # First path (main)
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 32)

        # Second path (skip connection)
        self.skip_fc1 = nn.Linear(784, 32)

        # Processing after concatenation
        self.concat_fc = nn.Linear(64, 128)
        self.skip_fc2 = nn.Linear(32, 128)

        # Final output layer
        self.final_fc = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input (28x28 -> 784)

        # First path (main)
        z = torch.relu(self.fc1(x))
        h = torch.relu(self.fc2(z))

        # Second path (skip connection)
        u = torch.relu(self.skip_fc1(x))

        # Concatenation
        q = torch.cat((h, u), dim=1)

        # Processing concatenated output
        v = torch.relu(self.concat_fc(q))

        # Further processing of skip connection
        k = torch.relu(self.skip_fc2(u))

        # Add the processed outputs
        d = v + k

        # Final output layer
        hat_y = self.final_fc(d)
        
        return hat_y

In [3]:
# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TwistedMNISTModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
# Training the model
num_epochs = 30

In [6]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 0.3796
Epoch 2, Loss: 0.1784
Epoch 3, Loss: 0.1344
Epoch 4, Loss: 0.1131
Epoch 5, Loss: 0.0971
Epoch 6, Loss: 0.0844
Epoch 7, Loss: 0.0772
Epoch 8, Loss: 0.0697
Epoch 9, Loss: 0.0626
Epoch 10, Loss: 0.0587
Epoch 11, Loss: 0.0542
Epoch 12, Loss: 0.0485
Epoch 13, Loss: 0.0468
Epoch 14, Loss: 0.0448
Epoch 15, Loss: 0.0425
Epoch 16, Loss: 0.0382
Epoch 17, Loss: 0.0364
Epoch 18, Loss: 0.0363
Epoch 19, Loss: 0.0322
Epoch 20, Loss: 0.0329
Epoch 21, Loss: 0.0302
Epoch 22, Loss: 0.0287
Epoch 23, Loss: 0.0288
Epoch 24, Loss: 0.0240
Epoch 25, Loss: 0.0282
Epoch 26, Loss: 0.0276
Epoch 27, Loss: 0.0248
Epoch 28, Loss: 0.0191
Epoch 29, Loss: 0.0253
Epoch 30, Loss: 0.0226


In [7]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [8]:
print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Save the trained model
torch.save(model.state_dict(), 'custom_mnist_model.pth')
print("Model saved successfully.")

Test Accuracy: 97.22%
Model saved successfully.
